<a href="https://colab.research.google.com/github/charoo-rumsan/community_tool_research/blob/main/pattern_recog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sample Dataset Generation

In [ ]:
import pandas as pd
import numpy as np

# Sample Upload 1: Headers with 'Amount'
data1 = {
    'Date': ['2023-01-01', '2023-01-02'],
    'Description': ['Lunch', 'Travel'],
    'Amount': ['$50.00', '$100.00']  # String with currency for realism
}
df1 = pd.DataFrame(data1)

# Sample Upload 2: Headers with 'Total'
data2 = {
    'Transaction Date': ['2023-02-01', '2023-02-02'],
    'Item': ['Office Supplies', 'Conference'],
    'Total': [75.50, 200.00]  # Numeric
}
df2 = pd.DataFrame(data2)

# Sample Upload 3: Headers with 'Cost' and 'Price' (multiple money fields for testing)
data3 = {
    'Date': ['2023-03-01', '2023-03-02'],
    'Vendor': ['Vendor A', 'Vendor B'],
    'Cost': [30, 40],
    'Price': [45.00, 55.00]
}
df3 = pd.DataFrame(data3)

# For testing, we'll use these DFs
uploads = [df1, df2, df3]

# Print samples
print("Sample Upload 1:\n", df1.head())
print("\nSample Upload 2:\n", df2.head())
print("\nSample Upload 3:\n", df3.head())

Sample Upload 1:
          Date Description   Amount
0  2023-01-01       Lunch   $50.00
1  2023-01-02      Travel  $100.00

Sample Upload 2:
   Transaction Date             Item  Total
0       2023-02-01  Office Supplies   75.5
1       2023-02-02       Conference  200.0

Sample Upload 3:
          Date    Vendor  Cost  Price
0  2023-03-01  Vendor A    30   45.0
1  2023-03-02  Vendor B    40   55.0


Method 1: Rule-Based Mapping

A class with a synonym dictionary that updates with user feedback

**__init__(self):**

Initializes the class with an empty dictionary self.synonyms. This dictionary will store known header-to-field mappings.

**map_field(self, header, df, manual_map=None):**

This is the main method for mapping a header.

It converts the input header to lowercase for case-insensitive matching.
If manual_map is provided (meaning the user manually specified a mapping), it adds this mapping to the self.synonyms dictionary and returns the manual_map with 100% confidence.

If no manual_map is given, it checks if the lowercase header exists in the self.synonyms dictionary. If found, it returns the corresponding mapped field with 100% confidence.

If the header is not found in the self.synonyms dictionary, it returns None and 0% confidence, indicating no mapping was found.

In [ ]:
class RuleBasedMapper:
    def __init__(self):
        self.synonyms = {}  # e.g., {'amount': 'expense_amount'}

    def map_field(self, header, df, manual_map=None):
        header_lower = header.lower()
        if manual_map:
            self.synonyms[header_lower] = manual_map
            return manual_map, 100  # Confidence 100% for manual
        if header_lower in self.synonyms:
            return self.synonyms[header_lower], 100
        return None, 0  # No map

# Test
mapper = RuleBasedMapper()

# Upload 1: Manual
print("Upload 1: Mapping 'Amount'")
map1, conf1 = mapper.map_field('Amount', uploads[0], manual_map='expense_amount')
print(f"Mapped to: {map1}, Confidence: {conf1}%")

# Upload 2: No map yet
print("\nUpload 2: Mapping 'Total'")
map2, conf2 = mapper.map_field('Total', uploads[1])
print(f"Mapped to: {map2}, Confidence: {conf2}%")  # None, suggest manual

# Simulate user feedback for 'Total'
mapper.map_field('Total', uploads[1], manual_map='expense_amount')

# Upload 3: Now maps if manual added, but for 'Cost'/'Price' still none
print("\nUpload 3: Mapping 'Cost'")
map3, conf3 = mapper.map_field('Cost', uploads[2])
print(f"Mapped to: {map3}, Confidence: {conf3}%")

Upload 1: Mapping 'Amount'
Mapped to: expense_amount, Confidence: 100%

Upload 2: Mapping 'Total'
Mapped to: None, Confidence: 0%

Upload 3: Mapping 'Cost'
Mapped to: None, Confidence: 0%


In [ ]:
# Manual testing with new data
print("\nManual Testing with New Data:")

# Create a new sample DataFrame
new_data = {
    'Invoice Number': ['INV-001', 'INV-002'],
    'Purchase Amount': [150.75, 220.00],
    'Due Date': ['2023-04-15', '2023-04-20']
}
new_df = pd.DataFrame(new_data)
print("New Sample Data:\n", new_df.head())

# Test mapping 'Purchase Amount' - should not be mapped initially
map_new1, conf_new1 = mapper.map_field('Purchase Amount', new_df['Purchase Amount'])
print(f"\nMapping 'Purchase Amount' (initial): {map_new1}, Confidence: {conf_new1}%")

# Manually map 'Purchase Amount'
mapper.map_field('Purchase Amount', new_df['Purchase Amount'], manual_map='expense_amount')
print("Manually mapped 'Purchase Amount' to 'expense_amount'")

# Test mapping 'Purchase Amount' again - should now be mapped
map_new2, conf_new2 = mapper.map_field('Purchase Amount', new_df['Purchase Amount'])
print(f"Mapping 'Purchase Amount' (after manual mapping): {map_new2}, Confidence: {conf_new2}%")

# Test mapping a non-money field 'Due Date'
map_new3, conf_new3 = mapper.map_field('Due Date', new_df['Due Date'])
print(f"Mapping 'Due Date': {map_new3}, Confidence: {conf_new3}%")


Manual Testing with New Data:
New Sample Data:
   Invoice Number  Purchase Amount    Due Date
0        INV-001           150.75  2023-04-15
1        INV-002           220.00  2023-04-20

Mapping 'Purchase Amount' (initial): expense_amount, Confidence: 79%
Manually mapped 'Purchase Amount' to 'expense_amount'
Mapping 'Purchase Amount' (after manual mapping): expense_amount, Confidence: 83%
Mapping 'Due Date': None, Confidence: 0%


Method 2: Fuzzy Matching with Heuristics

Use fuzzywuzzy for string similarity, plus heuristics like checking if column is numeric or has currency.

In [ ]:
# Install fuzzywuzzy if not already installed
!pip install fuzzywuzzy

from fuzzywuzzy import fuzz
import re

class FuzzyHeuristicMapper:
    def __init__(self):
        self.known_money_terms = ['amount', 'total', 'cost', 'price']  # Seed with common terms
        self.mappings = {}  # Learned mappings

    def is_money_column(self, col_data):
        # Heuristic: Numeric or has currency symbols
        if pd.api.types.is_numeric_dtype(col_data):
            return True
        return any(re.search(r'[\$\€\£]', str(val)) for val in col_data)

    def map_field(self, header, df_column, manual_map=None):
        header_lower = header.lower()
        if manual_map:
            self.mappings[header_lower] = manual_map
            self.known_money_terms.append(header_lower)
            return manual_map, 100
        # Fuzzy match to known terms
        scores = [fuzz.ratio(header_lower, term) for term in self.known_money_terms]
        max_score = max(scores) if scores else 0
        if max_score > 70 and self.is_money_column(df_column):  # Threshold
            return 'expense_amount', max_score
        return None, 0

# Test
mapper = FuzzyHeuristicMapper()

# Upload 1: Manual
print("Upload 1: Mapping 'Amount'")
map1, conf1 = mapper.map_field('Amount', uploads[0]['Amount'], manual_map='expense_amount')
print(f"Mapped to: {map1}, Confidence: {conf1}%")

# Upload 2: Fuzzy match to 'amount' (ratio ~50, but seed has 'total'—wait, 'total' in seed
print("\nUpload 2: Mapping 'Total'")
map2, conf2 = mapper.map_field('Total', uploads[1]['Total'])
print(f"Mapped to: {map2}, Confidence: {conf2}%")  # Should match since 'total' in seed, ratio 100

# Upload 3: 'Cost' in seed
print("\nUpload 3: Mapping 'Cost'")
map3, conf3 = mapper.map_field('Cost', uploads[2]['Cost'])
print(f"Mapped to: {map3}, Confidence: {conf3}%")
print("Mapping 'Price'")
map4, conf4 = mapper.map_field('Price', uploads[2]['Price'])
print(f"Mapped to: {map4}, Confidence: {conf4}%")

Upload 1: Mapping 'Amount'
Mapped to: expense_amount, Confidence: 100%

Upload 2: Mapping 'Total'
Mapped to: expense_amount, Confidence: 100%

Upload 3: Mapping 'Cost'
Mapped to: expense_amount, Confidence: 100%
Mapping 'Price'
Mapped to: expense_amount, Confidence: 100%


/usr/local/lib/python3.12/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Method 3: Semantic Similarity using NLP Embeddings

Use sentence-transformers for embeddings.

In [ ]:
from sentence_transformers import SentenceTransformer, util

class SemanticMapper:
    def __init__(self):
        self.model = SentenceTransformer('all-MiniLM-L6-v2')
        self.known_embeddings = {}  # {'term': embedding}
        self.mappings = {}  # {'term': 'expense_amount'}

    def map_field(self, header, df_column, manual_map=None):
        header_lower = header.lower()
        header_emb = self.model.encode(header_lower)
        if manual_map:
            self.known_embeddings[header_lower] = header_emb
            self.mappings[header_lower] = manual_map
            return manual_map, 100
        if not self.known_embeddings:
            return None, 0
        similarities = {term: util.cos_sim(header_emb, emb)[0][0].item() for term, emb in self.known_embeddings.items()}
        max_sim = max(similarities.values()) if similarities else 0
        if max_sim > 0.5:  # Threshold for suggestion/auto
            best_term = max(similarities, key=similarities.get)
            return self.mappings[best_term], int(max_sim * 100)
        return None, 0

# Test
mapper = SemanticMapper()

# Upload 1: Manual
print("Upload 1: Mapping 'Amount'")
map1, conf1 = mapper.map_field('Amount', uploads[0]['Amount'], manual_map='expense_amount')
print(f"Mapped to: {map1}, Confidence: {conf1}%")

# Upload 2: Semantic sim to 'amount'
print("\nUpload 2: Mapping 'Total'")
map2, conf2 = mapper.map_field('Total', uploads[1]['Total'])
print(f"Mapped to: {map2}, Confidence: {conf2}%")  # ~0.4-0.6 depending on model

# Simulate feedback
mapper.map_field('Total', uploads[1]['Total'], manual_map='expense_amount')

# Upload 3: Now sim to 'amount' or 'total'
print("\nUpload 3: Mapping 'Cost'")
map3, conf3 = mapper.map_field('Cost', uploads[2]['Cost'])
print(f"Mapped to: {map3}, Confidence: {conf3}%")  # High if similar

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Upload 1: Mapping 'Amount'
Mapped to: expense_amount, Confidence: 100%

Upload 2: Mapping 'Total'
Mapped to: expense_amount, Confidence: 67%

Upload 3: Mapping 'Cost'
Mapped to: expense_amount, Confidence: 51%


Method 4: Supervised Machine Learning Classification

Use scikit-learn RandomForest on features like TF-IDF of header + heuristics.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
import numpy as np

class MLMapper:
    def __init__(self):
        self.pipeline = Pipeline([
            ('tfidf', TfidfVectorizer()),
            ('clf', RandomForestClassifier())
        ])
        self.X = []  # Headers
        self.y = []  # Labels ('expense_amount' or 'other')

    def extract_features(self, header, df_column):
        # Simple: header text (TF-IDF will handle), plus heuristic as extra feature
        is_money = 1 if pd.api.types.is_numeric_dtype(df_column) else 0
        return header + f" money:{is_money}"

    def map_field(self, header, df_column, manual_map=None):
        features = self.extract_features(header, df_column)
        if manual_map:
            self.X.append(features)
            self.y.append(manual_map if manual_map == 'expense_amount' else 'other')
            if len(self.X) > 1:  # Need at least 2 for fit
                self.pipeline.fit(self.X, self.y)
            return manual_map, 100
        if not self.y:
            return None, 0
        pred = self.pipeline.predict([features])[0]
        prob = self.pipeline.predict_proba([features])[0][list(self.pipeline.classes_).index(pred)] * 100
        if pred == 'expense_amount':
            return pred, int(prob)
        return None, 0

# Test
mapper = MLMapper()

# Upload 1: Manual
print("Upload 1: Mapping 'Amount'")
map1, conf1 = mapper.map_field('Amount', uploads[0]['Amount'], manual_map='expense_amount')
print(f"Mapped to: {map1}, Confidence: {conf1}%")

# Add a non-money for balance
mapper.map_field('Date', uploads[0]['Date'], manual_map='other')

# Upload 2: Now predict
print("\nUpload 2: Mapping 'Total'")
map2, conf2 = mapper.map_field('Total', uploads[1]['Total'])
print(f"Mapped to: {map2}, Confidence: {conf2}%")

# Feedback
mapper.map_field('Total', uploads[1]['Total'], manual_map='expense_amount')

# Upload 3
print("\nUpload 3: Mapping 'Cost'")
map3, conf3 = mapper.map_field('Cost', uploads[2]['Cost'])
print(f"Mapped to: {map3}, Confidence: {conf3}%")

Upload 1: Mapping 'Amount'
Mapped to: expense_amount, Confidence: 100%

Upload 2: Mapping 'Total'
Mapped to: expense_amount, Confidence: 51%

Upload 3: Mapping 'Cost'
Mapped to: expense_amount, Confidence: 61%


Method 5: Hybrid Approach (Rules + ML + Feedback Loop)

Combine rule-based + fuzzy + ML; use rules first, then fuzzy, then ML if trained.

In [ ]:
# Assume classes from above: RuleBasedMapper, FuzzyHeuristicMapper, MLMapper

class HybridMapper:
    def __init__(self):
        self.rule_mapper = RuleBasedMapper()
        self.fuzzy_mapper = FuzzyHeuristicMapper()
        self.ml_mapper = MLMapper()

    def map_field(self, header, df_column, manual_map=None):
        if manual_map:
            # Feed to all
            self.rule_mapper.map_field(header, df_column, manual_map)
            self.fuzzy_mapper.map_field(header, df_column, manual_map)
            self.ml_mapper.map_field(header, df_column, manual_map)
            return manual_map, 100
        # Try rule first
        map_r, conf_r = self.rule_mapper.map_field(header, df_column)
        if conf_r > 90:
            return map_r, conf_r
        # Then fuzzy
        map_f, conf_f = self.fuzzy_mapper.map_field(header, df_column)
        if conf_f > 70:
            return map_f, conf_f
        # Then ML
        map_m, conf_m = self.ml_mapper.map_field(header, df_column)
        if conf_m > 50:
            return map_m, conf_m
        return None, 0

# Test similar to above
mapper = HybridMapper()

# Upload 1: Manual
print("Upload 1: Mapping 'Amount'")
map1, conf1 = mapper.map_field('Amount', uploads[0]['Amount'], manual_map='expense_amount')
print(f"Mapped to: {map1}, Confidence: {conf1}%")
mapper.map_field('Date', uploads[0]['Date'], manual_map='other')  # Balance

# Upload 2
print("\nUpload 2: Mapping 'Total'")
map2, conf2 = mapper.map_field('Total', uploads[1]['Total'])
print(f"Mapped to: {map2}, Confidence: {conf2}%")  # From fuzzy/rules

# Feedback
mapper.map_field('Total', uploads[1]['Total'], manual_map='expense_amount')

# Upload 3
print("\nUpload 3: Mapping 'Cost'")
map3, conf3 = mapper.map_field('Cost', uploads[2]['Cost'])
print(f"Mapped to: {map3}, Confidence: {conf3}%")

Upload 1: Mapping 'Amount'
Mapped to: expense_amount, Confidence: 100%

Upload 2: Mapping 'Total'
Mapped to: expense_amount, Confidence: 100%

Upload 3: Mapping 'Cost'
Mapped to: expense_amount, Confidence: 100%


Method 6: Advanced: Zero-Shot or Generative AI

Use a simple prompt-based approach with a mock LLM (since no openai, use rule as placeholder or torch-based, but for prototype, simulate with basic logic. In Colab, you can pip install transformers and use a small model like distilgpt2 for generation.)

In [ ]:
from transformers import pipeline

class GenerativeMapper:
    def __init__(self):
        self.generator = pipeline('text-generation', model='distilgpt2')
        self.known_examples = []

    def map_field(self, header, df_column, manual_map=None):
        if manual_map:
            self.known_examples.append(f"Header '{header}' maps to '{manual_map}'")
            return manual_map, 100
        prompt = "Map the header to a field. Known mappings: " + "; ".join(self.known_examples) + f". Now map '{header}' to expense_amount if it's money-related, with confidence."
        response = self.generator(prompt, max_length=50)[0]['generated_text']
        # Parse mock (in real, parse output)
        if 'expense_amount' in response:
            confidence = 90  # Mock parse
            return 'expense_amount', confidence
        return None, 0

# Test
mapper = GenerativeMapper()

# Upload 1: Manual
print("Upload 1: Mapping 'Amount'")
map1, conf1 = mapper.map_field('Amount', uploads[0]['Amount'], manual_map='expense_amount')
print(f"Mapped to: {map1}, Confidence: {conf1}%")

# Upload 2
print("\nUpload 2: Mapping 'Total'")
map2, conf2 = mapper.map_field('Total', uploads[1]['Total'])
print(f"Mapped to: {map2}, Confidence: {conf2}%")  # Model may infer

# Feedback and Upload 3 similar

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Upload 1: Mapping 'Amount'
Mapped to: expense_amount, Confidence: 100%

Upload 2: Mapping 'Total'
Mapped to: expense_amount, Confidence: 90%


In [ ]:
import difflib
from sentence_transformers import SentenceTransformer, util

# 1. Predefined dictionary (manual mapping)
canonical_map = {
    "expense_amount": ["Amount", "Total", "Cost", "Price"],
    "vendor_name": ["Vendor", "Supplier"]
}

# Build reverse lookup for exact matches
reverse_dict = {v.lower(): k for k, vals in canonical_map.items() for v in vals}

# 2. Embedding model (semantic similarity)
model = SentenceTransformer("all-MiniLM-L6-v2")

def map_column(col_name, threshold_fuzzy=0.8, threshold_embed=0.75):
    col_lower = col_name.lower()

    # Step 1: Exact dictionary match
    if col_lower in reverse_dict:
        return reverse_dict[col_lower], 1.0, "dictionary"

    # Step 2: Fuzzy match (using difflib)
    best_match = None
    best_score = 0
    for canonical_field, synonyms in canonical_map.items():
        for syn in synonyms:
            score = difflib.SequenceMatcher(None, col_name.lower(), syn.lower()).ratio()
            if score > best_score:
                best_score, best_match = score, canonical_field
    if best_score >= threshold_fuzzy:
        return best_match, best_score, "fuzzy"

    # Step 3: Embedding similarity
    candidates = list(canonical_map.keys())
    embeddings1 = model.encode([col_name], convert_to_tensor=True)
    embeddings2 = model.encode(candidates, convert_to_tensor=True)
    cosine_scores = util.pytorch_cos_sim(embeddings1, embeddings2)[0]

    best_idx = int(cosine_scores.argmax())
    best_score = float(cosine_scores[best_idx])
    if best_score >= threshold_embed:
        return candidates[best_idx], best_score, "embedding"

    # Step 4: Unknown → ask user feedback
    return "UNKNOWN", 0.0, "manual"

#  Test on sample dataset
columns = ["Amount", "Total", "Cost", "Price", "Totl", "Charges", "Value", "Vendor", "Supplier", "Vendor_Name"]

for col in columns:
    field, conf, method = map_column(col)
    print(f"{col:12s} → {field:15s} (conf: {conf:.2f}, via {method})")


Amount       → expense_amount  (conf: 1.00, via dictionary)
Total        → expense_amount  (conf: 1.00, via dictionary)
Cost         → expense_amount  (conf: 1.00, via dictionary)
Price        → expense_amount  (conf: 1.00, via dictionary)
Totl         → expense_amount  (conf: 0.89, via fuzzy)
Charges      → UNKNOWN         (conf: 0.00, via manual)
Value        → UNKNOWN         (conf: 0.00, via manual)
Vendor       → vendor_name     (conf: 1.00, via dictionary)
Supplier     → vendor_name     (conf: 1.00, via dictionary)
Vendor_Name  → vendor_name     (conf: 1.00, via embedding)
